# State Management with the GMAT API 
The state data in GMAT can be a bit confusing.  This notebook introduces the state variables as used for a GMAT Spacecraft, and provides some pointers on the manipulation of the state data.

## Prepare the GMAT Environment
Before the API can be used, it needs to be loaded into the Python system and initialized using a GMAT startup file.  This can be done from the GMAT bin folder by importing the gmatpy module, but using that approach tends to leave files in the bin folder that may annoy other users.  Running from an outside folder takes a few steps, which have been captured in the load_gmat.py file in the GMAT api folder.  After preparing the API for use (see the API "read me" file, api/API_README.txt), copy load_gmat.py into the folder you are using for Jupyter notebooks.  Then import it:

In [ ]:
from load_gmat import *

## Configure a Spacecraft
We'll need an object that provides the state.  Here's a basic spacecraft, along with a reference to the state data inside of the spacecraft:

In [ ]:
sat = gmat.Construct("Spacecraft","MySat")
iState = sat.GetState()

The state reference here, iState, operates on the member of the Spacecraft object that GMAT uses when running a simulation.  The "internal state," referenced by iState here, is the Earth-centered mean-of-J2000 equatorial representation of position and velocity of the spacecraft MySat.  The data is contained in a GmatState object:

In [ ]:
iState

GmatState objects are used to collect together an epoch and a vector of data.  These data can be accessed directly:

In [ ]:
print("The state epoch is ", iState.GetEpoch(), ", the state has ", iState.GetSize(), 
      " elements, and contains the data ", iState.GetState())

The data shown here is the default GmatState vector data for a spacecraft.  The epoch is January 1, 2000 at 12:00:00.000 in TAIModJulian time, or 21545.00000039794 in A.1 ModJulian time.  Note that GMAT uses A.1 Mod Julian as its internal epoch system.  The state has 6 elements.  The position and velocity data are filled in with the dummy entries -999.999. 
## Working with Cartesian and Keplerian Representations
A spacecraft in GMAT has a second collection of data: the state data for the spacecraft in the coordinate system set on the spacecraft.  These data are the spacecraft's "display state," named that way because they are the data displayed to the user.  Users interact with the display state similarly to the way they interact with the scripting language.  Data for a Keplerian state can be set using the SetField() method, as shown here: 

In [ ]:
sat.SetField("StateType", "Keplerian")
sat.SetField("SMA", 7015)
sat.SetField("ECC", 0.0011)
sat.SetField("INC", 98.6)
sat.SetField("RAAN", 75)
sat.SetField("AOP", 90)
sat.SetField("TA", 33.333)

At this point it can appear to the user that the data is set, but it really is not.  The spacecraft object cannot interpret the state data.  The data set using SetField needs more information than a spacecraft object can provide by itself.  Specifically, the spacecraft here does not have a connected coordinate system.  Cartesian state data set on the spacecraft does not have connections defining the coordinate origin, nor the structures needed to set the orientation of the axes defining directions.  Additionally, the spacecraft does not have the the gravitational constant needed to interpret Keplerian data.

In this uninitialized state, the spacecraft uses its GmatState buffer to hold the data entries.  We can see that the data is not yet fully populated by posting queries to the spacecraft:

In [ ]:
print("The internal state buffer just holds preinitialization data (Keplerian here):\n  ", iState.GetState())
print("but access to the Keplerian state shows that it is not correct:\n  ", sat.GetKeplerianState())

The Keplerian state data is not correct because the GMAT objects are not yet initialized.  Once we initialize the system, the Keplerian state will be correct, and the internal state will be updated to the EarthMJ2000Eq system.  The interobject connections necessary for these settings are made by calling the API's Initialize() function::

In [ ]:
gmat.Initialize()
print("The initialized internal state buffer is EarthMJ2000Eq:\n  ", iState.GetState())
print("and the Keplerian state is correct:\n  ", sat.GetKeplerianState())

Changes made to the state variables are now applied to the state as expected:

In [ ]:
sat.SetField("SMA", 8000)
print("Internal state:\n ", iState.GetState())
print("Cartesian:\n ", sat.GetCartesianState())
print("Keplerian:\n ", sat.GetKeplerianState())
print()
sat.SetField("INC", 45)
print("Internal state:\n ", iState.GetState())
print("Cartesian:\n ", sat.GetCartesianState())
print("Keplerian:\n ", sat.GetKeplerianState())
print()
sat.SetField("TA", 50)
print("Internal state:\n ", iState.GetState())
print("Cartesian:\n ", sat.GetCartesianState())
print("Keplerian:\n ", sat.GetKeplerianState())

## Changing Coordinate Systems
The previous section shows how to access Cartesian and Keplerian representations of the system.  In this section we will work with a couple of different coordinate systems: an Earth fixed coordinate system named "ECF" and accessed using the Python reference ecf, and a solar ecliptic system named "SolarEcliptic," referenced as sec.  These coordinate systems are built using the code 

In [ ]:
ecf = gmat.Construct("CoordinateSystem", "ECF", "Earth", "BodyFixed")
sec = gmat.Construct("CoordinateSystem", "SolarEcliptic", "Sun", "MJ2000Ec")

In this section, the spacecraft sat defined previously will be used with the Earth fixed coordinate system, and a copy of that spacecraft will be used with the solar ecliptic system.  GMAT's objects support a method, Copy(), that copies one object into another object of the same type.  Rather than set up a new spacecraft from scratch, we'll use that framework to get started by creating a new spacecraft and then setting the coordinate systems so that the original spacecraft uses the ECI coordinate system and the new spacecraft uses the solar ecliptic system.

In [ ]:
solsat = gmat.Construct("Spacecraft","SolarSat")
solsat.Copy(sat)

# Now set coordinate systems
sat.SetField("CoordinateSystem","ECF")
solsat.SetField("CoordinateSystem","SolarEcliptic")

We've reset the coordinate system names on the spacecraft at this point, but have yet to reset the associated objects because the Initialize() function that connects objects together has not been called since making the reassignment.  The data reflects this state of the system:

In [ ]:
# Show the data after setting the new coordinate systems, before initialization
print("The spacecraft ", sat.GetName(), " initialization state is ", sat.IsInitialized())
print("The internal state buffer:  ", iState.GetState())
print("The ECF Cartesian State:    ", sat.GetCartesianState())
print("The ECF Keplerian State:    ", sat.GetKeplerianState())
print()
print("The spacecraft ", solsat.GetName(), " initialization state is ", sat.IsInitialized())
print("The internal state buffer (SolarSat):  ", solsat.GetState().GetState())
print("The SolarEcliptic Cartesian State:     ", solsat.GetCartesianState())
print("The SolarEcliptic Keplerian State:     ", solsat.GetKeplerianState())

*Note that the initialization state reported here is a bug: resetting object references should toggle the initialization flag, but did not.*

Once we initialize the system, replacing the coordinate system references with the correct objects, the data is once again correct:

In [ ]:
# Connect the GMAT objects together
gmat.Initialize()

# And show the data in the new coordinate systems
print("The internal state buffer:  ", iState.GetState())
print("The ECF Cartesian State:    ", sat.GetCartesianState())
print("The ECF Keplerian State:    ", sat.GetKeplerianState())
print()
print("The internal state buffer (SolarSat):  ", solsat.GetState().GetState())
print("The SolarEcliptic Cartesian State:     ", solsat.GetCartesianState())
print("The SolarEcliptic Keplerian State:     ", solsat.GetKeplerianState())